## MUSIC RECOMMENDADOR

#### Schema 
If input song in list(hot100 titles):
Recommend another hot 100 song

Else:
Connect to spotipy with credentials
Search the input_song for the song ID on Spotify
With song_id find uri for song
Use URI to get the audio_features for the song
Put the audio_feeatures into a single row dataframe
Use scaler to scale the single row
Use the Kmeans model to predict the cluster for the input_song
Recommend another song from the same cluster

### Import all libraries to be used

In [ ]:
import random
import pickle
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
from sklearn.cluster import KMeans
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler

#### Connection to Spotify

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

secrets_file = open("secrets.txt","r")

string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

### Pickle save your kmeans model and scaler to use again

In [ ]:
#I DID NOT ATTEND WHEN WE HAD TO SAVE AN SCALER.PKL AND KMEANS.PKL FOR THIS LAB
#The open() function is used to open the file in binary mode ('rb'), which is required for reading binary files.
#The pickle.load() function is then used to deserialize the object from the file.

scaler = pickle.load(open('model.pkl','rb'))
kmeans = pickle.load(open('kmeans.pkl','rb'))

In [ ]:
filename = 'cd.pkl'
with open(filename, 'wb') as f:
    classification_dict = pickle.load(f)

In [ ]:
kmeans = pickle.load(open('model.pkl', 'rb'))
transformer = pickle.load(open('transformer.pkl','rb'))

### Import all needed .csv files and pickles

In [ ]:
clus_playlist = pd.read_csv("clus_playlist.csv")
top_100 = pd.read_csv("music_top100.csv")

In [ ]:
top_100.head()

In [ ]:
clus_playlist.columns

In [ ]:
top_100.head()

### Spotify Authetification

In [ ]:
secrets_dict={} #convert the clientid and clientsecrets into dictionaries
for line in string.split('\n'):
    if len(line) > 0: #if I don't have an empty line do next line.
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip() #.strip remove xtra spaces.

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [ ]:
clus_playlist

In [ ]:
song = input("Enter a song name: ")
is_song_in_list = clus_playlist['Title'].isin([song]).any()
client_id=secrets_dict['clientid']
client_secret=secrets_dict['clientsecret']

song = sp.search(q=song, type='track', limit=1)
uri = song['tracks']['items'][0]['uri']
audio_features = sp.audio_features(uri)

#### ITERATION LOOP WITH TOP 100 HOT PLAYLIST

In [ ]:
# the user to enter a song name
user_input = input("Enter a song name: ")

# Check if the user-provided song name is in the 'Title' column from top_100
if user_input in top_100['title'].values:
    print("Your song is in the Top 100!")

# Getting another song title from the top_100 (excluding the user's song)
other_songs = top_100[top_100['title'] != user_input]['title']
recommended_song = other_songs.sample(n=1).values[0]

    print("Additionally, I highly recommend this song from the Top 100 playlist:", recommended_song)
else:
    print("Your song is not in the Top 100.")

In [ ]:
audio_features

In [ ]:
audio_features_df = pd.DataFrame(audio_features, index=[0])

In [ ]:
selected_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
audio_song = audio_features_df[selected_columns]

In [ ]:
audio_song 

In [ ]:
audio_song = scaler.transform(audio_song)

In [ ]:
cluster_number = kmeans.predict(audio_song)

In [ ]:
list(cluster_number)[0]

In [ ]:
recommend_cluster = playlist_final[playlist_final['cluster']== list(cluster_number)[0]]

In [ ]:
recommend_cluster['name']

In [ ]:
recommend_cluster.sample()

### RECOMMENDATOR 

In [ ]:
def recommend_song():
    song = input("Enter a song name: ")
    #is_song_in_list = playlist_final['name'].isin([song]).any()
    client_id=secrets_dict['clientid']
    client_secret=secrets_dict['clientsecret']
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret'])) 
    if song in list(top_100['title']):
        recommended_song = top_100.sample(n=1)['title'].values[0]
        print("Your song is in the Top 100 list!")
        print("Here's another recommended song:", recommended_song)
    
    else:
        song = sp.search(q=song, type='track', limit=1)
        uri = song['tracks']['items'][0]['uri']
        audio_features = sp.audio_features(uri)
        audio_features_df = pd.DataFrame(audio_features, index=[0])
        selected_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
        audio_song = audio_features_df[selected_columns]
        audio_song = scaler.transform(audio_song) 
        cluster_number = kmeans.predict(audio_song)
        recommend_cluster = playlist_final[playlist_final['cluster']== list(cluster_number)[0]]
        rec = recommend_cluster.sample()
        
        print("Here's another recommended song:", rec[['name','artist']])

In [ ]:
recommend_song()